In [ ]:
!pip install pinecone-client
!pip install transformers datasets
!pip install cohere
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.4 MB/s eta 0:00:00
  Using cached multiprocess-0.70.17-py310-none-any.whl.metadata (7.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 M

In [2]:
pip install --upgrade pinecone-client

In [3]:
pip install pinecone-client datasets transformers torch cohere

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
INFO: pip is looking at multiple versions of pathos to determine which version is compatible with other requirements. This could take a while.
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.17
    Uninstalling multiprocess-0.70.17:
      Successfully uninstalled multiprocess-0.70.17
  Attempting uninstall: pathos
    Found existing installat

In [4]:
pip install --upgrade cohere

In [5]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [2]:
# RAG QA Bot Notebook


import os
import PyPDF2
from pinecone import Pinecone
from transformers import AutoTokenizer, AutoModel
import torch
import cohere
from google.colab import files

# Initialize Pinecone
pc = Pinecone(api_key="f4c8e22b-b1a2-4ecd-a1e5-f8afbf271048")
index = pc.Index("qa-bot-index")

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

# Initialize Cohere client
co = cohere.Client('9LA9uG3Mr6swUpOkd8PKBfyJ6EWyVTSvtZKufJ2S')

def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.squeeze().tolist()

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

def split_text(text, chunk_size=1000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def upload_and_process_pdf():
    print("Please upload your PDF file.")
    try:
        uploaded = files.upload()
        if not uploaded:
            print("No file was uploaded.")
            return None

        file_name = next(iter(uploaded))
        file_path = os.path.join(os.getcwd(), file_name)
        print(f"Processing {file_name}...")

        text = extract_text_from_pdf(file_path)
        chunks = split_text(text)

        print("Embedding and uploading chunks to Pinecone...")
        for i, chunk in enumerate(chunks):
            embedding = embed_text(chunk)
            doc_id = f"doc_{i}"
            index.upsert(vectors=[(doc_id, embedding, {"text": chunk})])

        print(f"Uploaded {len(chunks)} chunks to Pinecone.")
        return chunks
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

def generate_answer(query, context):
    prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    try:
        response = co.generate(
            model="command",
            prompt=prompt,
            max_tokens=500,
            temperature=0.7,
            stop_sequences=["Human:", "Context:"]
        )
        return response.generations[0].text.strip()
    except Exception as e:
        print(f"An error occurred while generating the answer: {str(e)}")
        return None

def main():
    chunks = upload_and_process_pdf()
    if not chunks:
        return

    print("\nExample queries:")
    example_queries = [
        "What is the main topic of the document?",
        "Summarise the document in detail.",
        "What are the key points and takeways?"
    ]

    for query in example_queries:
        print(f"\nQuery: {query}")
        query_embedding = embed_text(query)
        results = index.query(vector=query_embedding, top_k=3, include_metadata=True)

        if results['matches']:
            context = " ".join([match['metadata']['text'] for match in results['matches']])
            answer = generate_answer(query, context)
            print(f"Answer: {answer}")
        else:
            print("No relevant information found.")

    print("\nNow you can ask your own questions:")
    while True:
        query = input("\nEnter your question (or 'quit' to exit): ")
        if query.lower() == 'quit':
            break

        query_embedding = embed_text(query)
        results = index.query(vector=query_embedding, top_k=3, include_metadata=True)

        if results['matches']:
            context = " ".join([match['metadata']['text'] for match in results['matches']])
            answer = generate_answer(query, context)
            print(f"Answer: {answer}")
        else:
            print("No relevant information found.")

if __name__ == "__main__":
    main()

Please upload your PDF file.


Saving 001-Innodatatics AI Consulting Services-Projects (1).pdf to 001-Innodatatics AI Consulting Services-Projects (1).pdf
Processing 001-Innodatatics AI Consulting Services-Projects (1).pdf...
Embedding and uploading chunks to Pinecone...
Uploaded 41 chunks to Pinecone.

Example queries:

Query: What is the main topic of the document?
Answer: The main topic of the document is the success criteria and objectives of AI projects in the past work that Innodatatics has done.

Query: Summarise the document in detail?
Answer: The provided document details the success of various AI projects by outlining the criteria and objectives of each venture. From success in business, where a reduction of 75% in review time was achieved, to the accuracy of a high 80% in the automatic detection of pallets, the document highlights the benefits these projects have had for their respective industries. These case studies show how AI can be a useful tool for optimizing efficiency and accuracy across many fiel